In [163]:
import matplotlib.pyplot as plt
import math
import matplotlib.patches as patches
import numpy as np
import matplotlib
matplotlib.use('nbagg')
import matplotlib.animation as anm

from IPython.display import HTML # アニメーションが表示されないための処置

In [164]:
class World:
    def __init__(self, time_span, time_interval, debug=False):
        self.objects = [] #ここにロボットなどのオブジェクトを登録
        self.debug   = debug
        self.time_span = time_span  # シミュレーション時間
        self.time_interval = time_interval # シミュレーション刻み
    
    def append(self, obj): #オブジェクトを登録するための関数
        self.objects.append(obj)
    
    def draw(self):
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_subplot(111)
        ax.set_aspect('equal')
        ax.set_xlim(-5, 5)
        ax.set_ylim(-5, 5)
        ax.set_xlabel("X", fontsize=10)
        ax.set_ylabel("Y", fontsize=10)

        elems = []

        if self.debug:
            for i in range(1000):
                self.one_step(i, elems, ax) #デバッグ時はアニメーション表示させない
        else:
            self.anm = anm.FuncAnimation(fig, self.one_step, fargs=(elems, ax), frames=int(self.time_span/self.time_interval)+1, interval=int(self.time_interval*1000), repeat=False)
            plt.show()
            # LiveShareのParticipantsは描画されない
            # アニメーションが自動で再生されない
            plt.close()                         #アニメーションが表示されないための処置
            return HTML(self.anm.to_jshtml())   #アニメーションが表示されないための処置


    def one_step(self, i, elems, ax):
        while elems:
            elems.pop().remove()
        time_str = "t = %.2f[s]"%(self.time_interval*i) # 時刻として表示する文字列          
        elems.append(ax.text(-4.4, 4.5, time_str, fontsize=10))
        for obj in self.objects:
            obj.draw(ax, elems)
            if hasattr(obj, "one_step"): obj.one_step(self.time_interval) #　robotのone_step呼び出し


In [165]:
class IdealRobot:
    def __init__(self,pose,agent=None, sensor= None, color="black"):
        self.pose = pose    # 引数から姿勢の初期値を設定
        self.r = 0.2        # これは描画のためなので固定値（ロボットの丸の大きさ）
        self.color = color  # 引数から描画するときの色を設定
        self.agent = agent
        self.poses = [pose] # 軌跡の描画用
        self.sensor = sensor

    def draw(self, ax, elems):
        x, y, theta = self.pose     # 姿勢の変数を分解して3つの変数へ
        xn = x + self.r * math.cos(theta)    # ロボットの鼻先のx座標
        yn = y + self.r * math.sin(theta)    # ロボットの鼻先のy座標
        elems += ax.plot([x, xn],[y, yn], color=self.color) # ロボットの向きを示す線分の描画
        c = patches.Circle(xy=(x,y), radius=self.r, fill=False,color=self.color)
        elems.append(ax.add_patch(c)) # 上のpatches.Circleでロボットの胴体を示す円を作ってサブプロットへ登録

        self.poses.append(self.pose)
        elems += ax.plot([e[0] for e in self.poses], [e[1] for e in self.poses], linewidth=0.5, color="black")
        if self.sensor and len(self.poses) > 1:
            self.sensor.draw(ax, elems, self.poses[-2])

    def one_step(self, time_interval):
        if not self.agent: 
            return
        
        if self.sensor:
            obs = self.sensor.data(self.pose)  
        else:
            obs = None
        # obs = self.sensor ? self.sensor.data(self.pose) : None # C言語の三項演算子

        nu, omega = self.agent.decision(obs)
        self.pose = self.state_transition(nu, omega, time_interval, self.pose)
        


    @classmethod
    def state_transition(cls, nu, omega, time, pose): 
        theta0 = pose[2]
        if math.fabs(omega) < 1e-10: # 角速度がほとんどゼロのとき
            return pose + np.array([nu*math.cos(theta0),nu*math.sin(theta0),omega]) * time
        else:
            return pose + np.array([nu/omega*(math.sin(theta0+omega*time)-math.sin(theta0)), 
                                    nu/omega*(-math.cos(theta0+omega*time)+math.cos(theta0)),
                                    omega*time])

In [166]:
# IdealRobot.state_transition(0.1, 0.0, 1.0, np.array([0,0,0]).T)
# IdealRobot.state_transition(0.1, 10./180.*math.pi, 9.0, np.array([0,0,0]).T)
IdealRobot.state_transition(0.1, 10./180.*math.pi, 18.0, np.array([0,0,0]).T)

array([7.01670930e-17, 1.14591559e+00, 3.14159265e+00])

In [167]:
class Agent:
    def __init__(self, nu, omega):
        self.nu = nu
        self.omega = omega
    
    def decision(self, observation=None):
        return self.nu, self.omega
    
    

In [168]:
class Landmark:
    def __init__(self, x, y):
        self.pos = np.array([x,y]).T
        self.id = None
    
    def draw(self, ax, elems):
        c= ax.scatter(self.pos[0], self.pos[1], s=100, marker="*", label="landmarks", color="orange")
        elems.append(c)
        elems.append(ax.text(self.pos[0], self.pos[1], "id:" + str(self.id), fontsize=10))

In [169]:
class Map:
    def __init__(self):
        self.landmarks = []

    def append_landmark(self, landmark):
        landmark.id = len(self.landmarks) # 追加するランドマークにID付与
        self.landmarks.append(landmark)
    
    def draw(self, ax, elems):
        for lm in self.landmarks:
            lm.draw(ax, elems)

In [170]:
class IdealCamera:
    def __init__(self, env_map, distance_range=(0.5, 6.0), direction_range=(-math.pi/3,math.pi/3)):
        self.map = env_map
        self.lastdata = []

        self.distance_range = distance_range
        self.direction_range = direction_range

    def visible(self, polarpos):
        if polarpos is None:
            return False
        
        return self.distance_range[0] <= polarpos[0] <= self.distance_range[1]\
            and self.direction_range[0] <= polarpos[1] <= self.direction_range[1]

    
    def data(self, cam_pose):
        observed = []
        for lm in self.map.landmarks:
            z = self.observation_function(cam_pose, lm.pos)
            if self.visible(z):
                observed.append((z, lm.id))

        self.lastdata = observed
        return observed

    @classmethod
    def observation_function(cls, cam_pose, obj_pos): 
        diff = obj_pos - cam_pose[0:2] # cam_pose[0:2] = [cam_pose[0], cam_pose[1]], (diff_x, diff_y) の順
        phi = math.atan2(diff[1],diff[0])-cam_pose[2] # cam_pose[2] = \theta
        # -pi <= phi <= pi の中に収まるように調整
        while phi >= np.pi:
            phi -= 2*np.pi
        while phi < -np.pi:
            phi += 2*np.pi
        return np.array([np.hypot(*diff),phi]).T # p.hypot(*diff) = \sqrt{(m_{j,x}-x)^2 + (m_{j,y}-y)^2} と aten2(*) in Eq. (3.14) を返す
        # *はリストのアンパック

    def draw(self, ax, elems, cam_pose):
        for lm in self.lastdata:
            x, y, theta = cam_pose
            distance, direction = lm[0][0], lm[0][1]
            lx = x + distance * math.cos(direction + theta)
            ly = y + distance * math.sin(direction + theta)
            elems += ax.plot([x,lx], [y,ly], color="pink")
            
        


In [171]:
world = World(10,0.1, debug=False)

# 地図を生成して3つランドマークを追加
m = Map()
m.append_landmark(Landmark(2, -2))
m.append_landmark(Landmark(-1, -3))
m.append_landmark(Landmark(3, 3))
world.append(m)

# world.draw()
straight = Agent(0.2, 0.0)              # 直進する
circling = Agent(0.2, 10.0/180*math.pi) # 円を描く
stationary = Agent(0.0, 0.0)            # 静止

### 以下追加(3.2.2) ###
robot1 = IdealRobot(np.array([2,3,math.pi/6]).T, sensor=IdealCamera(m), agent=straight)          # ロボットのインスタンス生成（色を省略）
robot2 = IdealRobot(np.array([-2,-1,math.pi/5*6]).T, sensor=IdealCamera(m), agent=circling, color="red") # ロボットのインスタンス生成（色を指定）
robot3 = IdealRobot(np.array([0,0,0]).T, sensor=IdealCamera(m), agent=stationary, color="blue")          # ロボットのインスタンス生成（色を省略）

world.append(robot1)
world.append(robot2)
world.append(robot3)
world.draw()



<IPython.core.display.Javascript object>

In [172]:
cam = IdealCamera(m)
p=cam.data(robot2.pose)
print(robot2.pose)
print(p)

[-2.09321491 -2.77864643  5.55014702]
[(array([4.16661716, 0.92102083]), 0), (array([1.11539959, 0.53325976]), 1)]
